In [33]:
import pandas as pd
cdf_path = "raw/anes_timeseries_2024_csv_20250808.csv"
cdf = pd.read_csv(cdf_path)
cdf = cdf[cdf["V241700"] == -1]

/tmp/ipykernel_317014/4168556764.py:3: DtypeWarning: Columns (8,10,11,16,18,19,20,22,23,24,26,27,28,30,31,36,38,39,1448,1449,1450,1451,1452,1455,1456,1457,1458,1459,1463,1464,1465) have mixed types. Specify dtype option on import or set low_memory=False.
  cdf = pd.read_csv(cdf_path)


In [36]:
cdf["V241227x"].value_counts(dropna=False).sort_index()


V241227x
-9      28
-8       5
-4       2
 1    1082
 2     509
 3     582
 4     322
 5     592
 6     466
 7     946
Name: count, dtype: int64

In [46]:
items = {
    #"year": "VCF0004", # all 2024
    "weight": "V240107b",
}

recode_mapping = {
    "age": {"item": "V241458x", "mapping": {-2: pd.NA}},
    "female": {"item": "V241550", "mapping": {1: 0, 2: 1, 3: pd.NA, 0: pd.NA}},
    "race": {"item": "V241501x", "mapping": {1: "White", 2: "Black", 3: "Hispanic", 4: "Other", 5: "Other", 6: "Other", -8: pd.NA, -9: pd.NA}},
    "strong_republican": {"item": "V241227x", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, -1: pd.NA, -9: pd.NA}},
    "republican": {"item": "V241227x", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, -1: pd.NA, -9: pd.NA}},
    "democrat": {"item": "V241227x", "mapping": {1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, -1: pd.NA, -9: pd.NA}},
    "lean_republican": {"item": "V241227x", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, -1: pd.NA, -9: pd.NA}},
    "party_3_narrow": {"item": "V241227x", "mapping": {1: "D", 2: "D", 3: "I", 4: "I", 5: "I", 6: "R", 7: "R", 0: pd.NA}},
    "party_3_broad": {"item": "V241227x", "mapping": {1: "D", 2: "D", 3: "D", 4: "I", 5: "R", 6: "R", 7: "R", 0: pd.NA}},
    "conservative": {"item": "V241177", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, -1: pd.NA, -9: pd.NA}},
    "vote_rep_pres": {"item": "V242067", "mapping": {1: 0, 2: 1, -9: pd.NA, -8: pd.NA, -7: pd.NA, -6: pd.NA, -1: pd.NA, 4: pd.NA, 5: pd.NA, 6: pd.NA}},
    "prev_rep_pres": {"item": "V241104", "mapping": {1: 0, 2: 1, -9: pd.NA, -8: pd.NA, -7: pd.NA, -6: pd.NA, -1: pd.NA, 4: pd.NA, 5: pd.NA, 6: pd.NA}},
    "blacks_lazy": {"item": "V242542", "mapping": {-1: pd.NA, -5: pd.NA, -6: pd.NA, -7: pd.NA, -9: pd.NA}},
    "therm_blacks": {"item": "V242516", "mapping": {-4: pd.NA, -5: pd.NA, -6: pd.NA, -7: pd.NA, -9: pd.NA}},
    "therm_whites": {"item": "V241718", "mapping": {-4: pd.NA, -5: pd.NA, -6: pd.NA, -7: pd.NA, -9: pd.NA}},
    "therm_trans": {"item": "V241719", "mapping": {-4: pd.NA, -5: pd.NA, -6: pd.NA, -7: pd.NA, -9: pd.NA}},
    "therm_police": {"item": "V242150", "mapping": {-4: pd.NA, -5: pd.NA, -6: pd.NA, -7: pd.NA, -9: pd.NA}},
    "anti_imm": {"item": "V242227", "mapping": {-9: pd.NA, -8: pd.NA, -7: pd.NA, -6: pd.NA, -5: pd.NA}},
    "college": {"item": "V241465x", "mapping": {4: 1, 5: 1, 1: 0, 2: 0, 3: 0, -9: pd.NA, -8: pd.NA, -4: pd.NA, -2: pd.NA, -1: pd.NA}},
    "no_guar_jobs": {"item": "V241253", "mapping": {-1: pd.NA, -8: pd.NA, -9: pd.NA}},
    "distrust": {"item": "V241234", "mapping": {-9: pd.NA}}
}

calculated = {
    "therm_delta": lambda row: row["therm_whites"] - row["therm_blacks"],
    "age_group": lambda row: (
        pd.NA if pd.isna(row["age"]) else
        pd.NA if row["age"] == " " else
        "18-29" if 18 <= int(row["age"]) <= 29 else
        "30-44" if 30 <= int(row["age"]) <= 44 else
        "45-64" if 45 <= int(row["age"]) <= 64 else
        "65+" if int(row["age"]) >= 65 else
        pd.NA
    ),
    "resentment": lambda row: (
        pd.NA if row["V242300"] < 0 or row ["V242301"] < 0 or row["V242302"] < 0 or row["V242303"] < 0 else
        int(row["V242300"]) + int(row["V242303"]) - int(row["V242301"]) - int(row["V242302"])#+ 12 # if you want a 4-20 scale. 
    ),
    "race_edu_block": lambda row: (
        pd.NA if pd.isna(row["college"]) else
        "NonWhite" if row["V241501x"] in (2,3,4,5,6,7) else
        "WhiteCollege" if row["V241501x"] == 1 and row["college"] == 1 else
        "WhiteNonCollege" if row["V241501x"] == 1 and row["college"] == 0 else
        pd.NA
    ),
    "race_party_block": lambda row: (
        "WhiteRep" if row["V241501x"] == 1 and row["V241227x"] in (6,7) else
        "WhiteDem" if row["V241501x"] == 1 and row["V241227x"] in (1,2) else
        "WhiteInd" if row["V241501x"] == 1 and row["V241227x"] not in (1,2,6,7) else
        "NonWhite" if row["V241501x"] in (2,3,4,5,6,7) else
        pd.NA
    ),
    "vote_prev_vote": lambda row: (
        pd.NA if pd.isna(row["prev_rep_pres"]) or pd.isna(row["vote_rep_pres"]) else
        "Rep-Rep" if row["prev_rep_pres"] == 1 and row["vote_rep_pres"] == 1 else
        "Rep-Dem" if row["prev_rep_pres"] == 1 and row["vote_rep_pres"] == 0 else
        "Dem-Rep" if row["prev_rep_pres"] == 0 and row["vote_rep_pres"] == 1 else
        "Dem-Dem" if row["prev_rep_pres"] == 0 and row["vote_rep_pres"] == 0 else
        pd.NA
    )
}

In [47]:
anls = cdf.copy()

for item_name, var in items.items():
    anls[item_name] = anls[var]

for new_var, spec in recode_mapping.items():
    #anls[new_var] = anls[spec["item"]].map(spec["mapping"])
    # we should make it so that unmapped values pass through directly
    anls[new_var] = anls[spec["item"]].replace(spec["mapping"])

for new_var, func in calculated.items():
    anls[new_var] = anls.apply(func, axis=1)

all_keys = list(items.keys()) + list(recode_mapping.keys()) + list(calculated.keys())
anls = anls[all_keys]
anls

,weight,age,female,race,strong_republican,republican,democrat,lean_republican,party_3_narrow,party_3_broad,...,anti_imm,college,no_guar_jobs,distrust,therm_delta,age_group,resentment,race_edu_block,race_party_block,vote_prev_vote
0,.710864874498513,50,0,Hispanic,1,1,0,1,R,R,...,3,1,1,3,-61,45-64,-8,NonWhite,NonWhite,Rep-Rep
1,2.38378558802356,41,1,Other,0,0,0,0,I,I,...,4,0,2,3,-51,30-44,-4,NonWhite,NonWhite,<NA>
2,.808992408255851,44,1,White,0,0,1,0,D,D,...,1,0,4,3,-91,30-44,8,WhiteNonCollege,WhiteDem,Dem-Dem
3,.291739965421631,45,0,Other,0,1,0,1,R,R,...,3,0,4,4,-51,45-64,-2,NonWhite,NonWhite,<NA>
4,.222012175174604,80,0,White,0,0,0,1,I,R,...,5,1,1,4,-51,65+,-4,WhiteCollege,WhiteInd,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,.554677396396131,42,1,White,0,0,0,1,I,R,...,3,1,1,2,-86,30-44,-7,WhiteCollege,WhiteInd,<NA>
5515,.0757582475292791,80,1,Hispanic,0,0,1,0,I,D,...,3,1,4,4,-51,65+,2,NonWhite,NonWhite,Dem-Dem
5516,,<NA>,1,<NA>,0,0,0,0,I,I,...,<NA>,1,<NA>,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5518,,<NA>,1,Black,0,0,0,0,I,I,...,<NA>,0,4,3,<NA>,<NA>,<NA>,NonWhite,NonWhite,<NA>
